# Profitable App Profiles for the App Store and Google Play Markets¶
### Here is my project when learning Python fundamentals for Data Analysis.
This project is about finding mobile app profiles that are profitable for the App Store and Google Play markets.
Our goal for this project is to analyze data to help our developers understand what type of apps are likely to attract more users.


# Opening and Exploring the Data
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

A data set containing data about approximately ten thousand Android apps from Google Play. You can download the data set directly from this [link](https://www.kaggle.com/lava18/google-play-store-apps/download).<br>
A data set containing data about approximately seven thousand iOS apps from the App Store. You can download the data set directly from this [link](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/download).

Let's start by opening the two data sets and then continue with exploring the data.

In [12]:
from csv import reader

# Define a function to open data set
def open_dataset(csv_file):
    '''
    - Takes one parameter:
    csvfile is expected to be a name of dataset
    - Open a dataset and return data_header and dataset
    '''
    opened_file = open(csv_file, encoding='utf8') # return io.TextIOWrapper Object)
    read_file = reader(opened_file) # return csv.reader Object 
    dataset = list(read_file) # transform csv.reader to list
    data_header = dataset[0]
    dataset = dataset[1:]
    return data_header, dataset


### The Google Play data set ###
android_header, android = open_dataset('googleplaystore.csv')

### The App Store data set ###
ios_header, ios = open_dataset('AppleStore.csv')

To make it easier to explore the two data sets, we'll first write a function named explore_data() that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any data set.

In [13]:
def explore_data(dataset, start, end, rows_and_columns=False):
    '''
    + Takes in four parameters
        - dataset is expected to be a list of lists.
        - start and end are both expected to be integers and represent
        the starting and the ending indices of a slice from the data set
        - rows_and_columns is expected to be a Boolean and has False as a default argument
    + Slices the data set using dataset[start:end]
    + Loops through the slice, and for each iteration, prints a row
    and adds a new line after that row
    + Print the number of rows and columns if rows_and_columns is True
        - dataset shouldn't have a header row.
    '''
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line between rows
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [14]:
### Exploring android data ###
print(android_header)
print('\n')
explore_data(android, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


We see that the Google Play data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.

In [15]:
### Exploring ios data ###
print(ios_header)
print('\n')
explore_data(ios, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


We have 7197 iOS apps in this data set, and the columns that seem interesting are: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'. Not all column names are self-explanatory in this case, but details about each column can be found in the data set [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home).

## Before beginning our analysis
We need to:<br/>
- Detect inaccurate data, and correct or remove it.<br/>
- Detect duplicate data, and remove the duplicates.<br/>

We only build **free to download apps** and **for English-speaking audience**.<br/>
This means that we'll need to:<br/>
- Remove non-English apps like 日本語ソフト<br/>
- Remove apps that aren't free.

# Deleting Wrong Data
The Google Play data set has a dedicated discussion section, and we can see that one of the [discussions outlines](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) an error for row 10472. Let's print this row and compare it against the header and another row that is correct.

In [17]:
print(android[10472])  # incorrect row
print('\n')
print(android_header)  # header
print('\n')
print(android[0])      # correct row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


The row 10472 corresponds to the app Life Made WI-Fi Touchscreen Photo Frame, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5 (as mentioned in the (discussions section)[https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015], this problem is caused by a missing value in the 'Category' column). As a consequence, we'll delete this row.

In [18]:
print(len(android))
del android[10472]
print(len(android))

10841
10840


# Removing Duplicate Entries
Simple check if there is duplicate apps on our two data sets.

In [20]:
print('Android')
for app in android:
    name = app[0]
    if name == 'Facebook':
        print(app)

print('\nIOS')

for app in ios:
    name = app[0]
    if name == 'Facebook':
        print(app)
        

Android
['Facebook', 'SOCIAL', '4.1', '78158306', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'August 3, 2018', 'Varies with device', 'Varies with device']
['Facebook', 'SOCIAL', '4.1', '78128208', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'August 3, 2018', 'Varies with device', 'Varies with device']

IOS


### Write a function to check how many cases where an app occurs more than once:

In [41]:
def counting_duplicate_app(dataset, dataset_name):
    duplicate_apps = []
    unique_apps = []
    
    for app in dataset:
        # check duplicate by name of the apps
        name = app[0]
        if name in unique_apps:
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)
    if duplicate_apps:
        print("{} has duplicate apps, here are the details".format(dataset_name))
        print('Number of duplicate apps: ', len(duplicate_apps))
        # print few names of apps that are duplicate
        print('Examples of duplicate apps: ', duplicate_apps[:5])
    else:
        print("{} data set has no duplicate apps\n\n".format(dataset_name))

In [42]:
# IOS
counting_duplicate_app(ios, "IOS")
# Android
counting_duplicate_app(android, "Android")

IOS data set has no duplicate apps


Android has duplicate apps, here are the details
Number of duplicate apps:  1181
Examples of duplicate apps:  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. One thing we could do is remove the duplicate rows randomly, but we could probably find a better way.

If you examine the rows we printed three cells above for the Facebook app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show that the data was collected at different times. We can use this to build a criterion for keeping rows. We won't remove rows randomly, but rather we'll keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable the ratings.

On the previous screen, we looped through the Google Play data set and found that there are 1,181 duplicates. After we remove the duplicates, we should be left with 9,659 rows:

In [43]:
# Because only Google Play data has duplicate data
print('Expted length: ', len(android) - 1181)

Expted length:  9659


We will
- Create a dictionary to store a unique name of app and the value is the highest value of **reviews** of the app.
- Use that dictionary to create a new data set (for each app, only select the entry with the highest value of **reviews**)

In [44]:
reviews_max = {}

for each in android:
    name = each[0]
    value_reviews = float(each[3])
        
    if name in reviews_max and reviews_max[name] < value_reviews:
        reviews_max[name] = value_reviews
    elif name not in reviews_max:
        reviews_max[name] = value_reviews

In [46]:
print('Expected length:', len(android) - 1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


Now, let's use the `reviews_max` dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. In the code cell below:
- We start by initializing two empty lists, `android_clean` and `already_added`.
- We loop through the android data set, and for every iteration:
 - We isolate the name of the app and the number of reviews.
 - We add the current row (app) to the android_clean list, and the app name (name) to the `already_added` list **if**:
  - The number of reviews of the current app matches the number of reviews of that app as described in the `reviews_max` dictionary
  - And the name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for `reviews_max[name]` == `n_reviews`, we'll still end up with duplicate entries for some apps.

In [47]:
android_clean = []
already_added = []

for each in android:
    name = each[0]
    value_reviews = float(each[3])
    # if statement:
    # get name of this app, 
    # check if value of reviews is the highest values
    # and check name to make sure it don't be duplicate again
    
    if (reviews_max[name] == value_reviews) and (name not in already_added):
        android_clean.append(each)
        already_added.append(name)

print(len(android_clean))

9659


If you explore the data sets enough, you'll notice the names of some of the apps suggest they are not directed toward an English-speaking audience. Below, we see a couple of examples from both data sets:


In [48]:
print(ios[813][1])
print(ios[6731][1])

print(android_clean[4412][0])
print(android_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜
中国語 AQリスニング
لعبة تقدر تربح DZ


We're not interested in keeping these kind of apps, so we'll remove them. One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.

We built this function below, and we use the built-in ord() function to find out the corresponding encoding number of each character.

In [49]:
def is_english(name):
    for char in name:
        if ord(char) > 127:
            return False
    return True

In [50]:
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
False
False


The algorithm seems work fine, but wrong at the test 3 and 4.
If we're going to use that function, we'll lose useful data since many English apps will be incorrectly labeled as a non-English.
We define a threshold, if there is more than 3 character that non-English char (over 127), we will mark it as non-English app.

In [51]:
def is_english(name):
    over_127 = 0
    for char in name:
        if ord(char) > 127:
            over_127 += 1
            
    if over_127 > 3:
        return False
    else:
        return True

In [53]:
# Test the function
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
True
True


The function is still not perfect, and very few non-English apps might get past our filter, but this seems good enough at this point in our analysis — I don't want to spend too much time on optimization at this point.

Below, we use the is_english() function to filter out the non-English apps for both data sets:

In [54]:
english_app_android = []
english_app_ios = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        english_app_android.append(app)
        
for app in ios:
    name = app[1]
    if is_english(name):
        english_app_ios.append(app)


In [56]:
explore_data(english_app_android, 0, 3, True)
print("\n")
explore_data(english_app_ios, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

We can see that we're left with 9614 Android apps and 6183 iOS apps.

### So far in the data cleaning process, we:
- Removed inaccurate data
- Removed duplicate app entries
- Removed non-English apps

Last step is remove non-free app.

# Isolating the Free Apps
As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [57]:
android_final_data = []
ios_final_data = []

# Android
for app in english_app_android:
    price = app[7]
    
    if price == '0':
        android_final_data.append(app)

# IOS
for app in english_app_ios:
    price = app[4]
    
    if price == '0.0':
        ios_final_data.append(app)       
        
# check the leng of each data set:

print(len(android_final_data))
print(len(ios_final_data))

8864
3222


We have 8864 android apps and 3222 ios apps, which should be enough for our analysis.

As we mentioned in the introduction, our revenue is influenced by the number of users using our app so we can get money by ads.
And our app will be launch in both App Store and CH Play, so we have to find the successful "factor" in both markets.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

- Build a minimal Android version of the app, and add it to Google Play.
- If the app has a good response from users, we then develop it further.
- If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app like learning languages.

Let's begin the analysis by getting a sense of the most common genres for each market.

For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

We'll build two functions we can use to analyze the frequency tables:<br/>
- One function to generate frequency tables that show percentages
- Another function we can use to display the percentages in a descending order

In [58]:
def frequency_table(dataset, index):
    frequency_table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in frequency_table:
            frequency_table[value] += 1
        else:
            frequency_table[value] = 1
    
    frequency_table_percentage = {}
    
    for key in frequency_table:
        frequency_table_percentage[key] = (frequency_table[key] / total) * 100
    
    return frequency_table_percentage
    

def display_table(dataset, index):
    table = frequency_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [59]:
display_table(ios_final_data, -5) # prime_genre

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Analyze the frequency table you generated for the prime_genre column of the App Store data set.<br/>
- The most common genre is Games(58.16%) and the runner-up is Entertainment(close to 8%).
- Other patterns are Photo & Video(close to 5%), Education(3.66%), Social Networking(3.29%), Shopping..
- My general impression is the most apps designed for entertainment

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

In [60]:
display_table(android_final_data, 1) # category

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

![ch play genre](ch_play_genre.png "ch play genre")


Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:

In [61]:
display_table(android_final_data, -4) # Genres

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

# Most Popular Apps by Genre on the App Store
One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre.<br/>
For the Google Play data set, we can find this information in the Installs column, but this information is missing for the App Store data set.<br/>
As a workaround, we'll take the total number of `user ratings` as a proxy, which we can find in the `rating_count_tot`app.<br/>
Let's start with calculating the average number of user ratings per app genre on the App Store. To do that, we'll need to: <br/>
- Isolate the apps of each genre.
- Sum up the user ratings for the apps of that genre.
- Divide the sum by the number of apps belonging to that genre (not by the total number of apps).

Below, we calculate the average number of user ratings per app genre on the App Store:

In [64]:
prime_genres = frequency_table(ios_final_data, -5)

for genre in prime_genres:
    total = 0
    len_genre = 0
    for app in ios_final_data:
        
        genre_app = app[-5]
        
        if genre_app == genre:
            number_ratings = float(app[5])
            total += number_ratings
            len_genre += 1
            
    avg_number_ratings = total / len_genre
    print(genre, ' : ', avg_number_ratings)

Social Networking  :  71548.34905660378
Photo & Video  :  28441.54375
Games  :  22788.6696905016
Music  :  57326.530303030304
Reference  :  74942.11111111111
Health & Fitness  :  23298.015384615384
Weather  :  52279.892857142855
Utilities  :  18684.456790123455
Travel  :  28243.8
Shopping  :  26919.690476190477
News  :  21248.023255813954
Navigation  :  86090.33333333333
Lifestyle  :  16485.764705882353
Entertainment  :  14029.830708661417
Food & Drink  :  33333.92307692308
Sports  :  23008.898550724636
Book  :  39758.5
Finance  :  31467.944444444445
Education  :  7003.983050847458
Productivity  :  21028.410714285714
Business  :  7491.117647058823
Catalogs  :  4004.0
Medical  :  612.0


Let's take a look at some genres that have the most user reviews

In [65]:
for app in ios_final_data:
    if app[-5] == 'Navigation':
        print(app[1], ' : ', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic  :  345046
Google Maps - Navigation & Transit  :  154911
Geocaching®  :  12811
CoPilot GPS – Car Navigation & Offline Maps  :  3582
ImmobilienScout24: Real Estate Search in Germany  :  187
Railway Route Search  :  5


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [67]:
for app in ios_final_data:
    if app[-5] == 'Social Networking':
        print(app[1], ' : ', app[5])

Facebook  :  2974676
Pinterest  :  1061624
Skype for iPhone  :  373519
Messenger  :  351466
Tumblr  :  334293
WhatsApp Messenger  :  287589
Kik  :  260965
ooVoo – Free Video Call, Text and Voice  :  177501
TextNow - Unlimited Text + Calls  :  164963
Viber Messenger – Text & Call  :  164249
Followers - Social Analytics For Instagram  :  112778
MeetMe - Chat and Meet New People  :  97072
We Heart It - Fashion, wallpapers, quotes, tattoos  :  90414
InsTrack for Instagram - Analytics Plus More  :  85535
Tango - Free Video Call, Voice and Chat  :  75412
LinkedIn  :  71856
Match™ - #1 Dating App.  :  60659
Skype for iPad  :  60163
POF - Best Dating App for Conversations  :  52642
Timehop  :  49510
Find My Family, Friends & iPhone - Life360 Locator  :  43877
Whisper - Share, Express, Meet  :  39819
Hangouts  :  36404
LINE PLAY - Your Avatar World  :  34677
WeChat  :  34584
Badoo - Meet New People, Chat, Socialize.  :  34428
Followers + for Instagram - Follower Analytics  :  28633
GroupMe  :  

The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

In [68]:
for app in ios_final_data:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. 

We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.



# Most Popular Apps by Genre on Google Play
For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [69]:
display_table(android_final_data, 5) # the Installs columns

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to find out which app genres attract the most users, and we don't need perfect precision with respect to the number of users.<br/>

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [72]:
categories = frequency_table(android_final_data, 1)

for category in categories:
    total = 0
    len_category = 0
    
    for app in android_final_data:
        if app[1] == category:
            n_installs = app[5]
            n_installs = n_installs.replace('+', '')
            n_installs = n_installs.replace(',', '')
            total += float(n_installs)
            len_category += 1
            
    avg_n_installs = total / len_category
    print(category, ' : ', avg_n_installs)

ART_AND_DESIGN  :  1986335.0877192982
AUTO_AND_VEHICLES  :  647317.8170731707
BEAUTY  :  513151.88679245283
BOOKS_AND_REFERENCE  :  8767811.894736841
BUSINESS  :  1712290.1474201474
COMICS  :  817657.2727272727
COMMUNICATION  :  38456119.167247385
DATING  :  854028.8303030303
EDUCATION  :  1833495.145631068
ENTERTAINMENT  :  11640705.88235294
EVENTS  :  253542.22222222222
FINANCE  :  1387692.475609756
FOOD_AND_DRINK  :  1924897.7363636363
HEALTH_AND_FITNESS  :  4188821.9853479853
HOUSE_AND_HOME  :  1331540.5616438356
LIBRARIES_AND_DEMO  :  638503.734939759
LIFESTYLE  :  1437816.2687861272
GAME  :  15588015.603248259
FAMILY  :  3695641.8198090694
MEDICAL  :  120550.61980830671
SOCIAL  :  23253652.127118643
SHOPPING  :  7036877.311557789
PHOTOGRAPHY  :  17840110.40229885
SPORTS  :  3638640.1428571427
TRAVEL_AND_LOCAL  :  13984077.710144928
TOOLS  :  10801391.298666667
PERSONALIZATION  :  5201482.6122448975
PRODUCTIVITY  :  16787331.344927534
PARENTING  :  542603.6206896552
WEATHER  :  50

Let's check what kind of app in 'COMMUNICATION' genre and see why it has the most install number

In [73]:
for app in android_final_data:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      ):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Viber Messenger : 500,000,000+


As we can see, Whatsapp, Messenger, Skype, Google Chorme? Gmail, Hangouts...
If we remove app that has over 100m install number?

In [74]:
under_100_m = []

for app in android_final_data:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386

It is roughly reduced 10 times (38456119.167247385)
Let's check if ios_data has the same problem.


In [76]:
under_100_m = []

for app in ios_final_data:
    n_ratings = app[5]
    if (app[-5] == 'Social Networking') and (float(n_ratings) < 60000):
        under_100_m.append(float(n_ratings))
        
sum(under_100_m) / len(under_100_m)



8856.71590909091

So with the old value (71548.34905660378) we can see that if we remove all the outliers(big tech, popular app) our data is reduced ~ 10 times. We could go through two data sets to apply this step but as our goal for this project is using python fundamentals, we'll stop here and build a profile app.

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.
After the data has been analyzed, I see that Education and Games are two genres we could focus on to create an app profile. Why not combine two of them?

I recommend we create a learning language app. This is a safe plan because learning languages is always a need for everyone. And we will create an learning app that has game in it like quiz, complete the lesson to unlock (images/games/fact about that country).
<br/>
Some common features: 
- Camera: Scan and translate
- Dictionary for multi language: Data (example, explanation about word), and comment feature for users to discuss.
- Flash card: Create and learn
- Lessons based on level. If you want to reach new lesson, you have to passed the test of the current level.
<br/>


How about getting pay/money?
- We'll set the limit for those above features. 
- Users can unlock it by watching ads (let's say 2 ads for 1 hour premium, 1 time a day) or pay it.
